# Load libraries

In [184]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Load the CSV file

In [185]:
data = pd.read_csv("account_daily_class28_pl_5000_dd_25.csv")
data.shape

(60327, 36)

# Use the last column as y, assign it to label variable

In [186]:
label = data.iloc[: , -1].name

In [187]:
data.head()

,User_ID,TIME,BALANCE_PREV,BALANCE,DEPOSIT,CREDIT,EQUITY,Close_PL,Open_PL,MARGIN,MARGIN_FREE,Margin_Ratio,Sharpe_Ratio_By_Balance_1W,Sharpe_Ratio_By_Ticket_1W,Sharpe_Ratio_By_Balance_2W,Sharpe_Ratio_By_Ticket_2W,Sharpe_Ratio_By_Balance_1M,Sharpe_Ratio_By_Ticket_1M,Profit_trades,Loss_trades,Long_trades,Short_trades,Number_of_Ticket,Ratio1,Ratio2,Ratio3,Ratio4,Ratio5,Ratio6,Account_Score,Combined_Score,Profit_Trades_Ratio,Long_Trades_Ratio,Total_PL,AccountDown_Ratio,PL_5000_DD_25
0,200000062,2021-02-24 21:59:59,2.76,74.58,300.0,0,74.58,-228.18,0.00,0.000000,74.580000,0.000000,-0.38,-3.14,-0.27,-3.14,-0.18,-3.14,0,3,0,3,3,1.17,0.71,1.83,2.02,1.09,0.36,1.520181,3.070766,0.000000,0.0,-1.0,0.0,0
1,200000062,2021-02-25 21:59:59,74.58,247.11,0.0,0,301.76,172.53,54.65,147.105473,154.654527,0.487492,-0.37,-0.17,-0.26,-0.17,-0.18,-0.17,8,6,0,14,14,1.00,0.65,1.83,1.78,1.08,0.36,1.189500,2.117310,0.571429,0.0,-1.0,0.0,0
2,200000062,2021-02-26 21:59:59,247.11,269.56,0.0,0,373.43,22.45,103.87,197.133384,176.296616,0.527899,-0.37,-0.17,-0.26,-0.17,-0.18,-0.17,12,7,0,19,19,0.97,0.62,1.83,1.76,1.08,0.36,1.100562,1.936989,0.631579,0.0,-1.0,0.0,0
3,200000062,2021-02-27 21:59:59,269.56,269.56,0.0,0,373.43,0.00,103.87,197.133384,176.296616,0.527899,-0.37,-0.17,-0.26,-0.17,-0.18,-0.17,12,7,0,19,19,0.97,0.62,1.83,1.76,1.08,0.36,1.100562,1.936989,0.631579,0.0,-1.0,0.0,0
4,200000062,2021-02-28 21:59:59,269.56,269.56,0.0,0,373.43,0.00,103.87,197.133384,176.296616,0.527899,-0.37,-0.17,-0.26,-0.17,-0.18,-0.17,12,7,0,19,19,0.97,0.62,1.83,1.76,1.08,0.36,1.100562,1.936989,0.631579,0.0,-1.0,0.0,0


In [188]:
len(y_test)

12066

# Drop irrelevant features for training

In [189]:

data_train = data.drop(['User_ID', 'TIME', 'BALANCE_PREV', 'BALANCE', 'DEPOSIT', 'CREDIT', 'Close_PL', 'Open_PL', 'MARGIN', 'MARGIN_FREE', 'Sharpe_Ratio_By_Ticket_1W', 'Sharpe_Ratio_By_Ticket_2W', 'Sharpe_Ratio_By_Ticket_1M','Number_of_Ticket', 'Profit_trades', 'Loss_trades', 'Long_trades', 'Short_trades', 'Total_PL', 'AccountDown_Ratio', label], axis=1)

#good result below
#data_train = data.drop(['User_ID', 'TIME', 'BALANCE_PREV', 'BALANCE', 'DEPOSIT', 'CREDIT', 'Close_PL', 'Open_PL', 'MARGIN', 'MARGIN_FREE', 'Sharpe_Ratio_By_Ticket_1W', 'Sharpe_Ratio_By_Ticket_2W', 'Sharpe_Ratio_By_Balance_1W','Sharpe_Ratio_By_Balance_1M', 'Sharpe_Ratio_By_Ticket_1M','Number_of_Ticket', 'Profit_trades', 'Loss_trades', 'Long_trades', 'Short_trades', 'Total_PL', 'AccountDown_Ratio', label], axis=1)

In [190]:
data_train.head()

,EQUITY,Margin_Ratio,Sharpe_Ratio_By_Balance_1W,Sharpe_Ratio_By_Balance_2W,Sharpe_Ratio_By_Balance_1M,Ratio1,Ratio2,Ratio3,Ratio4,Ratio5,Ratio6,Account_Score,Combined_Score,Profit_Trades_Ratio,Long_Trades_Ratio
0,74.58,0.000000,-0.38,-0.27,-0.18,1.17,0.71,1.83,2.02,1.09,0.36,1.520181,3.070766,0.000000,0.0
1,301.76,0.487492,-0.37,-0.26,-0.18,1.00,0.65,1.83,1.78,1.08,0.36,1.189500,2.117310,0.571429,0.0
2,373.43,0.527899,-0.37,-0.26,-0.18,0.97,0.62,1.83,1.76,1.08,0.36,1.100562,1.936989,0.631579,0.0
3,373.43,0.527899,-0.37,-0.26,-0.18,0.97,0.62,1.83,1.76,1.08,0.36,1.100562,1.936989,0.631579,0.0
4,373.43,0.527899,-0.37,-0.26,-0.18,0.97,0.62,1.83,1.76,1.08,0.36,1.100562,1.936989,0.631579,0.0


# Split data to create training and testing data for X and y, 

In [191]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data[label], test_size = 0.2, random_state=0)
#X_train, X_test = train_test_split(data, test_size=0.2, random_state=0)

# Now, combine X_train and y_train to be used to create a balanced training data

In [192]:
X_y_train = pd.concat((X_train,y_train), axis=1)

# Filter items by class values (1 and 0), and count the number of those value is 1

In [193]:
positive_train = X_y_train[X_y_train[label]==1]
number_positive = len(positive_train)
negative_train = X_y_train[X_y_train[label]==0]

In [194]:
y_test_dict = y_test.to_dict()

In [195]:
to_predict = y_test_dict.copy()

In [196]:
for (k,v) in to_predict.items():
  to_predict[k]= 0

In [197]:
y_test_indices = list(to_predict.keys())
y_test_values = list(to_predict.values())

# Sample the negative training data with the same number as that of the positive train data

In [198]:
negative_numb_factor = 1.5

Linear_SV_Classifier = SVC(kernel='linear',  verbose=1)  # Linear Kernel SVC
#poly_SV_Classifier = SVC(kernel='poly', degree=3, verbose=1)  # Polynomial Kernel SVC
rbf_SV_Classifier= SVC(kernel='rbf', gamma=0.5, verbose=1)


for rand_state_num in range(100):
  selected_negative_train = negative_train.sample(n=int(number_positive*negative_numb_factor), random_state=rand_state_num)
  X_y_train = pd.concat((positive_train,selected_negative_train), axis=0)
  X_train = X_y_train.drop([label], axis=1)
  y_train = X_y_train[label]
  SC = StandardScaler()
  X_train = SC.fit_transform(X_train)
  X_test = SC.fit_transform(X_test)
  #Linear_SV_Classifier.fit(X=X_train, y=y_train)
  #poly_SV_Classifier.fit(X=X_train, y=y_train)
  rbf_SV_Classifier.fit(X=X_train, y=y_train)

  #y_pred_1 = Linear_SV_Classifier.predict(X_test)

  #y_pred_2 = poly_SV_Classifier.predict(X_test)

  y_pred_3 = rbf_SV_Classifier.predict(X_test)

  for i in range(len(y_pred_3)):
    #y_test_values[i] += y_pred_1[i]
    #y_test_values[i] += y_pred_2[i]
    y_test_values[i] += y_pred_3[i]

  cm_lin = confusion_matrix(y_pred=y_pred_1, y_true=y_test)

  print("Confusion Matrix for Linear Classifier: ")
  print(cm_lin)

  #cm_poly = confusion_matrix(y_pred=y_pred_2, y_true=y_test)

  #print("Confusion matrix for Polynomial classifier with degree 3: ")
  #print(cm_poly)

  cm_rbf = confusion_matrix(y_pred=y_pred_3, y_true=y_test)

  print("Confusion matrix for RBF: ")
  print(cm_rbf)

  acc_lin = 100*np.trace(cm_lin)/np.sum(cm_lin)
  #acc_poly = 100*np.trace(cm_poly)/np.sum(cm_poly)
  acc_rbf = 100*np.trace(cm_rbf)/np.sum(cm_rbf)
  #print("Accuracy for linear: {}%, Polynomial (deg=3): {}% and RBF: {}% ".format(acc_lin, acc_poly, acc_rbf))
  print("Accuracy for linear: {}%,  RBF: {}% ".format(acc_lin, acc_rbf))


# good result as below
#selected_negative_train = negative_train.sample(n=int(number_positive*1.5), random_state=111)



[LibSVM]Confusion Matrix for Linear Classifier: 
[[9302 2680]
 [  46   38]]
Confusion matrix for RBF: 
[[9677 2305]
 [  27   57]]
Accuracy for linear: 77.40759157964528%,  RBF: 80.67296535720206% 
[LibSVM]Confusion Matrix for Linear Classifier: 
[[9302 2680]
 [  46   38]]
Confusion matrix for RBF: 
[[10436  1546]
 [   36    48]]
Accuracy for linear: 77.40759157964528%,  RBF: 86.88877838554616% 
[LibSVM]Confusion Matrix for Linear Classifier: 
[[9302 2680]
 [  46   38]]
Confusion matrix for RBF: 
[[9783 2199]
 [  27   57]]
Accuracy for linear: 77.40759157964528%,  RBF: 81.55146693187469% 
[LibSVM]Confusion Matrix for Linear Classifier: 
[[9302 2680]
 [  46   38]]
Confusion matrix for RBF: 
[[9102 2880]
 [  27   57]]
Accuracy for linear: 77.40759157964528%,  RBF: 75.90750870213824% 
[LibSVM]Confusion Matrix for Linear Classifier: 
[[9302 2680]
 [  46   38]]
Confusion matrix for RBF: 
[[8889 3093]
 [  24   60]]
Accuracy for linear: 77.40759157964528%,  RBF: 74.16708105420189% 
[LibSVM]Con

In [199]:
y_test_values

[1,
 0,
 53,
 2,
 0,
 79,
 0,
 50,
 0,
 61,
 51,
 28,
 4,
 0,
 27,
 19,
 5,
 2,
 2,
 90,
 93,
 4,
 0,
 7,
 0,
 15,
 38,
 0,
 0,
 3,
 80,
 0,
 21,
 44,
 4,
 0,
 0,
 0,
 4,
 99,
 1,
 13,
 20,
 1,
 0,
 0,
 100,
 0,
 5,
 0,
 0,
 100,
 0,
 81,
 0,
 1,
 0,
 2,
 34,
 100,
 35,
 3,
 0,
 0,
 0,
 2,
 0,
 5,
 61,
 90,
 0,
 0,
 31,
 0,
 1,
 8,
 7,
 15,
 6,
 0,
 4,
 16,
 14,
 0,
 0,
 63,
 11,
 0,
 5,
 63,
 23,
 0,
 0,
 32,
 52,
 1,
 1,
 0,
 23,
 0,
 18,
 1,
 0,
 0,
 6,
 0,
 3,
 3,
 14,
 0,
 0,
 39,
 0,
 37,
 0,
 0,
 3,
 33,
 0,
 0,
 77,
 0,
 0,
 46,
 0,
 53,
 88,
 16,
 0,
 0,
 30,
 0,
 93,
 0,
 0,
 0,
 29,
 100,
 5,
 53,
 2,
 0,
 0,
 0,
 0,
 0,
 18,
 87,
 6,
 41,
 66,
 6,
 63,
 1,
 0,
 0,
 75,
 71,
 7,
 11,
 0,
 0,
 0,
 7,
 28,
 0,
 39,
 88,
 0,
 6,
 3,
 2,
 100,
 52,
 96,
 0,
 0,
 0,
 12,
 60,
 35,
 39,
 0,
 5,
 23,
 45,
 0,
 3,
 17,
 8,
 34,
 88,
 40,
 27,
 0,
 4,
 52,
 0,
 0,
 69,
 3,
 82,
 18,
 72,
 48,
 0,
 0,
 97,
 0,
 23,
 0,
 0,
 10,
 0,
 6,
 3,
 32,
 11,
 0,
 24,
 0,
 14,
 73,
 0,
 12,
 0,

In [200]:
y_test

58265    0
26446    0
49158    0
40611    0
49202    0
        ..
58000    0
6968     0
7713     0
28167    0
14674    0
Name: PL_5000_DD_25, Length: 12066, dtype: int64

In [201]:
threshold = 50
for i in range(len(y_test_values)):
  if y_test_values[i] > threshold:
    y_test_values[i] = 1
  else:
    y_test_values[i] = 0

In [202]:
y_test_values

[0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,


In [203]:
final_y_preidct_arr = np.array(y_test_values)

In [204]:
cm_combined = confusion_matrix(y_pred=final_y_preidct_arr, y_true=y_test)

In [205]:
print(cm_combined)

[[10254  1728]
 [   28    56]]
